# *Siglo XXI*
##Challenge Data engineer 
###Azure

Contexto 

Lectura de Datos desde una Base de Datos:

- Conéctate a una base de datos SQL en Azure utilizando PySpark.
- Realiza una consulta para extraer un conjunto de datos específico.
- Realiza transformaciones en los datos según sea necesario.

Escribe los datos de la parte 1.2 en un Azure Blob Storage en formato parquet y con las 
particiones que consideres adecuadas.

Para realizar este ejercicio se utilizó la siguiente Base de datos:
    
GO

	CREATE DATABASE Comics;

GO

	Use Comics;

GO

Create table Superheroes (

	Personaje varchar(20),

	Nombre_Alter_Ego varchar(20),

	Edad INT,

	Altura_en_CM Int,

	Peso_en_kg Decimal(5,2),

	Superpoder varchar(255),
  
	Ciudad varchar(20)
);

GO

Insert into Superheroes Values

	('Batman','Bruno Díaz',30,188,95,'Ser Millonario','Ciudad Gótica'),

	('Spider Man','Peter Parker',18,178,76,'Movimientos araña','New york'),

	('Wonder Woman','Diana Prince',900,183,75,'Lazo salvaje','Islas Amazonas'),

	('Super Hijitus','Hijitus',null,null,null,'Sombrero Mágico','Trulalá'),

	('Wasp Woman','Janet van Dyne',26,162,50,'Controla bichos','New York'),

	('Hulk','Bruce Banner',47,300,540,'Ser verde y romper todo','New York'),

	('Catwoman','Selina Kyle',23,174,59.4,'Movimientos Gato','Ciudad Gótica'),

	('Radioactive man','Claude Kane III',null,null,null,'Super Fuerza','Springfield'),

	('Arrow','Oliver Queen',29,180,83.9,'Tirar flechas','Star City'),

	('The Invisible Woman','Susan Storm',27,170,54,'Ser Invisible','New York');
 
 GO

### **ETL realizado con pySpark**

#### Ingesta de datos desde una base de datos Azure SQL Database

Se importan las librerias

In [ ]:
%run "/sigloXXI/Librerias"

Se realiza la conexión con Azure SQL Database, con la base de datos "Comic" y se trae la tabla Superheroes

In [ ]:
jdbcHostname = "HOSTNAME" 
jdbcPort = <PORT> 
jdbcDatabase = "<DATABASE>" 
jdbcUsername = "<USER>" 
jdbcPassword = "<PASSWORD>"  
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}"

Se crea un dataframe a partir de la tabla Superheroes

In [ ]:
df = spark.read.format("jdbc").option("url", jdbcUrl).option("dbtable", "<TABLE>").load()
display(df)

### **Persistensia de Datos en DataLakeGen2**

Parte 2: Integración de Servicios en Azure 

Escribe los datos de la parte 1.2 en un Azure Blob Storage en formato parquet y con las 
particiones que consideres adecuadas. 

In [ ]:
# Configuracion de la conexión 
configs = {"fs.azure.account.auth.type":"OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "<CLIENT_ID>",
           "fs.azure.account.oauth2.client.secret": "<SECRET>",
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<ENDPOINT>/oauth2/token"}
dbutils.fs.mount(
    source = "abfss://<CONTAINER>@<STORAGE_ACCOUNT>.dfs.core.windows.net/",
    mount_point = "/mnt/<STORAGE_ACCOUNT>/<CONTAINER>",
    extra_configs = configs
)

Se escribe el DataFrame en formato Parquet particionado por el campo Personaje

In [ ]:
# Se calcula la fecha y hora actual
date_file = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Ruta destino mas fecha y hora actual
ruta_parquet = "/mnt/<STORAGE_ACCOUNT>/<CONTAINER>" + date_file

# Escribe el DataFrame en formato Parquet con partición
df.write.partitionBy("Personaje").parquet(ruta_parquet, mode="overwrite")

print("Datos procesados almacenados en Azure Data Lake Storage")